In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import json
import os
import pandas as pd
from tqdm import tqdm
requests_dir = '../data/input/requests'
response_dir = '../data/input/responses'


import sys
sys.path.append('../utils')
import helpers as h

## Combining all routes in a single df.
We'll output a single dataframe that has all route requests read and parsed into a single df.

In [ ]:
def read_request(route_id, ymd, idx_file):
    """
    Gepaste docstring in het goede formaat.
    Als je dit in het goede formaat doet kan je meteen documentatie maken
    https://realpython.com/python-project-documentation-with-mkdocs/

    Functie van Tim
    """
    
    folder_requests = os.path.join('..', 'data', 'input', 'requests', f"{route_id}-{ymd}")
    if not os.path.exists(folder_requests):
        return None

    file_request = os.listdir(folder_requests)[idx_file]
    file_path_request = os.path.join(folder_requests, file_request)
    print(file_path_request)
    with open(file_path_request, 'r') as f:
        request = json.load(f)
    print('OK')
    rows = list()
    for task in request['tasks']:
        # TODO: Dit kan efficienter met een specifieke methode (pd.explode, json to dataframe zaken)
        if task['id'] == 'E1':
            continue ## TODO: Also retain this task!
        row = {'id' : int(task['id']),
                    'lat' : task['address']['latitude'],
                    'long' : task['address']['longitude'],
                    'start_time' : task['timeWindow']['from'],
                    'end_time' : task['timeWindow']['till']}
        rows.append(row)

    return pd.DataFrame(rows).sort_values(by = 'id')
def get_route_dataframe(route_id, ymd, idx_file): 
    """
        Simple one linter that uses read_reques and sort_request to
        get the route dataframe from the JSON file, can be used as
        input for other functions. 
    """
    df_request = read_request(route_id, ymd, idx_file)
    #df_request = sort_request(df_request, route_id, ymd, idx_file)
    return df_request




In [33]:

def json_request_to_df(path): 
    """
        variation on Tim's function 'read_request', we'll not let a funciton decide
        what index file (last integer iin the filename) to read. in stead we do this
        in the df itself applying filters where needed. Gives us more control in case
        a json response for a hardcoded idx in read_requst is invalid. 

        ARGUMENTS: 
            path = str = Fully qualified path to a json file

        RETURNS: 
            pandas dataframe
    """
    with open(path, 'r') as f:
        request = json.load(f)
    rows = list()
    #meta: 
    file_uuid = request['id']
    configurationName = request['configurationName']
    for task in request['tasks']:
        # TODO: Dit kan efficienter met een specifieke methode (pd.explode, json to dataframe zaken)
        if task['id'] == 'E1':
            continue ## TODO: Also retain this task!
        row = { 
                'file_uuid' : file_uuid,
                'configurationName' : configurationName,
                'id' : int(task['id']),
                'lat' : task['address']['latitude'],
                'long' : task['address']['longitude'],
                'start_time' : task['timeWindow']['from'],
                'end_time' : task['timeWindow']['till']
                }
        rows.append(row)

    return pd.DataFrame(rows).sort_values(by = 'id')

#reading requests: 
internal_id = 1     #using internal_id we can quikly comminicate with each other what precise file we want to look at for debugging/app-feature design. 
all_content = []    # the final dataframe (build once all data is read)
for dir in tqdm(os.listdir(requests_dir)):
    if dir.endswith(".txt"):
        continue
    route_id = dir.split('-')[0]
    route_date = h.routedatestring_to_date(dir.split('-')[1])
    contents = os.listdir(os.path.join(requests_dir, dir))
    for file in contents:
        if file.endswith('.json'): 
            fn = file.rstrip('.json').split('-')
            idblock_1 = fn[2]
            idblock_2 = fn[3]
            idblock_3 = fn[4]
            fq_path = os.path.join(requests_dir, dir, file)
            content_of_file = json_request_to_df(fq_path)
            #enabling us to identify rows belong to a specific file on the drive. 
            content_of_file['dir'] = dir
            content_of_file['route_id'] = route_id
            content_of_file['route_date'] = route_date
            content_of_file['idblock_1'] = idblock_1
            content_of_file['idblock_2'] = idblock_2
            content_of_file['idblock_3'] = idblock_3
            content_of_file['internal_id'] = internal_id
            internal_id+=1
            all_content.append(content_of_file)


100%|██████████| 3726/3726 [05:28<00:00, 11.35it/s]


In [35]:
df_requests = pd.concat(all_content)

In [41]:
os.makedirs('../data/intermediate', exist_ok=True)
df_requests.to_csv('../data/intermediate/requests.csv', index=False)
